In [62]:
import cx_Oracle
import pandas as pd
import numpy as np

db_username_read = open('db_username.txt')
db_username = db_username_read.read()
db_username_read.close()

db_password_read = open('db_password.txt')
db_password = db_password_read.read()
db_password_read.close()

db_host_read = open('db_host.txt')
db_host = db_host_read.read()
db_host_read.close()

connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=db_host)
cursor = connection.cursor()

# Wyciągnięcie danych z bazy
query = """
SELECT gs.game_name, p.platform_name, pub.publisher_name, d.developer_name, g.genre_name, r.rating_name, gs.na_sales, gs.eu_sales, gs.jp_sales, gs.other_sales, gs.critic_score, gs.critic_count, gs.user_score, gs.user_count, gs.release_date
FROM game_sales gs
JOIN platforms p ON gs.platform_id = p.platform_id
JOIN publishers pub ON gs.publisher_id = pub.publisher_id
JOIN developers d ON gs.developer_id = d.developer_id
JOIN genres g ON gs.genre_id = g.genre_id
JOIN ratings r ON gs.rating_id = r.rating_id
"""
cursor.execute(query)

columns = ['Game_Name', 'Platform', 'Publisher', 'Developer', 'Genre', 'Rating', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Release_Date']
data = cursor.fetchall()
df = pd.DataFrame(data, columns=columns)

df.head()

,Game_Name,Platform,Publisher,Developer,Genre,Rating,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Release_Date
0,New Super Luigi U,WiiU,Nintendo,Nintendo,Platform,E,1250000,620000,180000,180000,77,59,7.9,288,2013
1,Tetris DS,DS,Nintendo,Nintendo,Puzzle,E,630000,50000,1350000,80000,84,56,8.7,44,2006
2,Classic NES Series: Super Mario Bros.,GBA,Nintendo,Nintendo,Platform,E,0,0,1390000,30000,84,14,8.6,44,2004
3,Pikmin 2,GC,Nintendo,Nintendo,Strategy,E,480000,130000,560000,30000,90,54,9.1,137,2004
4,Mario vs. Donkey Kong,GBA,Nintendo,Nintendo,Puzzle,E,680000,250000,210000,20000,81,43,8.0,31,2004


In [63]:
df = df[(df['Platform'] == 'PS3') | (df['Platform'] == 'PS4') | (df['Platform'] == 'X360') | (df['Platform'] == 'XOne') | (df['Platform'] == 'Wii') | (df['Platform'] == 'WiiU') | (df['Platform'] == 'PC')]
df = df[df['Release_Date'] > 2003]
df['NA_Sales'] = np.log1p(df['NA_Sales'])
df['EU_Sales'] = np.log1p(df['EU_Sales'])
df['JP_Sales'] = np.log1p(df['JP_Sales'])
df['Other_Sales'] = np.log1p(df['Other_Sales'])
df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

In [64]:
devs = len(df['Developer'].unique())
pubs = len(df['Publisher'].unique())
rats = len(df['Rating'].unique())
gens = len(df['Genre'].unique())
plats = len(df['Platform'].unique())
print('Developers: ', devs,'\nPublishers: ', pubs,'\nRatings: ', rats,'\nGenres: ', gens,'\nPlatforms: ', plats)

Developers:  762 
Publishers:  165 
Ratings:  5 
Genres:  12 
Platforms:  7


In [65]:
df = pd.get_dummies(data=df, columns=['Platform', 'Genre', 'Rating'])

In [66]:
X = df[['Release_Date', 'Critic_Score', 'Critic_Count',
       'User_Score', 'User_Count', 'Platform_PC', 'Platform_PS3',
       'Platform_PS4', 'Platform_Wii', 'Platform_WiiU', 'Platform_X360',
       'Platform_XOne', 'Genre_Action', 'Genre_Adventure', 'Genre_Fighting',
       'Genre_Misc', 'Genre_Platform', 'Genre_Puzzle', 'Genre_Racing',
       'Genre_Role-Playing', 'Genre_Shooter', 'Genre_Simulation',
       'Genre_Sports', 'Genre_Strategy', 'Rating_E', 'Rating_E10+', 'Rating_M',
       'Rating_RP', 'Rating_T', 'NA_Sales']]

y = df[['Total_Sales']]

print(X.shape)
print(y.shape)

(3174, 30)
(3174, 1)


In [67]:
y = np.ravel(y)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

grid_search_lr = GridSearchCV(LinearRegression(), {}, scoring='neg_mean_squared_error',  cv=5)
grid_search_lr.fit(X_train, Y_train)

# Calculate the RMSE for the best cross-validation score
lr_best_cross_val_score = (np.sqrt(-grid_search_lr.best_score_))
print("Best cross-validation score: {:.2f}".format(lr_best_cross_val_score))
y_pred = grid_search_lr.predict(X_test)

# Obliczenie RMSE dla zestawu testowego
test_rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print("Test RMSE score: {:.2f}".format(test_rmse))

# Obliczenie R^2 dla zestawu testowego
r2_score_value = r2_score(Y_test, y_pred)
print("Test R^2 score: {:.2f}".format(r2_score_value))

Best cross-validation score: 6.82
Test RMSE score: 6.96
Test R^2 score: 0.69


In [69]:
predicted_Total_Sales = pd.DataFrame({'Actual Total_Sales': Y_test, 'Predicted Total_Sales': y_pred})

predicted_Total_Sales['Difference'] = predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']
predicted_Total_Sales['Difference_in_%'] = np.abs(predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']) / ((predicted_Total_Sales['Actual Total_Sales'] + predicted_Total_Sales['Predicted Total_Sales']) / 2) * 100
print(predicted_Total_Sales.sample(10))

     Actual Total_Sales  Predicted Total_Sales  Difference  Difference_in_%
50            37.323309              33.962632    3.360676         9.428722
438           10.308986              26.531739  -16.222753        88.069674
235           52.354690              37.948498   14.406191        31.906274
422           55.959145              52.656290    3.302856         6.081743
504           39.058397              30.879447    8.178950        23.389195
609           31.974950              30.783722    1.191228         3.796217
417           30.116094              27.238455    2.877639        10.034564
54            21.254000              29.133450   -7.879450        31.275447
192           38.652936              42.540166   -3.887230         9.575272
509           35.463063              35.683610   -0.220547         0.619978


In [70]:
param_grid_rf = {'n_estimators': [3, 10, 30, 50, 70], 'max_features': [2,4,6,8,10,12], 'max_depth': [2, 3, 5, 7, 9]}

In [71]:
from sklearn.ensemble import RandomForestRegressor

grid_search_rf = GridSearchCV(RandomForestRegressor(), param_grid_rf, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, Y_train)

# Calculate the RMSE for the best cross-validation score
rf_best_cross_val_score = np.sqrt(-grid_search_rf.best_score_)
print("Cross-validated RMSE score: {:.2f}".format(rf_best_cross_val_score))

y_pred = grid_search_rf.predict(X_test)

# Calculate the RMSE for the test set
test_rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print("Test RMSE score: {:.2f}".format(test_rmse))

# Calculating R^2 score for the test set
r2_score_value = r2_score(Y_test, y_pred)
print("Test R^2 score: {:.2f}".format(r2_score_value))

Cross-validated RMSE score: 5.83
Test RMSE score: 6.02
Test R^2 score: 0.77


In [72]:
predicted_Total_Sales = pd.DataFrame({'Actual Total_Sales': Y_test, 'Predicted Total_Sales': y_pred})

predicted_Total_Sales['Difference'] = predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']
predicted_Total_Sales['Difference_in_%'] = np.abs(predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']) / ((predicted_Total_Sales['Actual Total_Sales'] + predicted_Total_Sales['Predicted Total_Sales']) / 2) * 100
print(predicted_Total_Sales.sample(10))

     Actual Total_Sales  Predicted Total_Sales  Difference  Difference_in_%
311           49.743804              48.932174    0.811631         1.645042
84            32.846036              27.553086    5.292950        17.526579
220           42.508598              45.731538   -3.222940         7.304930
232           46.510763              27.839098   18.671665        50.226496
518           19.807100              21.044292   -1.237192         6.057036
190           20.985738              25.006359   -4.020621        17.483966
441           30.339225              26.056924    4.282301        15.186501
321           21.059845              27.641068   -6.581223        27.027101
224           33.760122              36.631130   -2.871008         8.157288
399           10.308986              15.536356   -5.227370        40.451159


In [73]:
param_grid_gbr = {'n_estimators': [200, 225, 250, 275], 'max_features': [6, 8, 10, 12], 'max_depth': [5, 7, 9]}

In [74]:
from sklearn.ensemble import GradientBoostingRegressor

grid_search_gbr = GridSearchCV(GradientBoostingRegressor(), param_grid_gbr, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')
grid_search_gbr.fit(X_train, Y_train)

# Calculate the RMSE for the best cross-validation score
gbr_best_cross_val_score = np.sqrt(-grid_search_gbr.best_score_)
print("Cross-validated RMSE score: {:.2f}".format(gbr_best_cross_val_score))

y_pred = grid_search_gbr.predict(X_test)


# Calculate the RMSE for the test set
test_rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print("Test RMSE score: {:.2f}".format(test_rmse))

# Calculating R^2 score for the test set
r2_score_value = r2_score(Y_test, y_pred)
print("Test R^2 score: {:.2f}".format(r2_score_value))

Cross-validated RMSE score: 5.71
Test RMSE score: 5.87
Test R^2 score: 0.78


In [75]:
predicted_Total_Sales = pd.DataFrame({'Actual Total_Sales': Y_test, 'Predicted Total_Sales': y_pred})

predicted_Total_Sales['Difference'] = predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']
predicted_Total_Sales['Difference_in_%'] = np.abs(predicted_Total_Sales['Actual Total_Sales'] - predicted_Total_Sales['Predicted Total_Sales']) / ((predicted_Total_Sales['Actual Total_Sales'] + predicted_Total_Sales['Predicted Total_Sales']) / 2) * 100
print(predicted_Total_Sales.sample(10))

     Actual Total_Sales  Predicted Total_Sales  Difference  Difference_in_%
155           19.113978              18.208933    0.905045         4.849807
129           10.596660              16.856861   -6.260201        45.605815
511           36.519109              35.050227    1.468882         4.104781
358           38.227956              24.714270   13.513686        42.939968
104           36.470610              43.210803   -6.740193        16.917856
608           36.320505              34.432819    1.887685         5.335962
619           19.807100              12.689117    7.117983        43.808073
83            19.807100              15.168924    4.638176        26.522033
147           44.062551              40.972135    3.090416         7.268601
51            23.544672              33.570008  -10.025336        35.105987
